<a href="https://colab.research.google.com/github/mornevwyk/fCC/blob/main/regression-health-cost/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
data_copy = dataset.copy()

In [ ]:
categories = ['sex','children','smoker','region']
for cat in categories:
  dataset[cat] = dataset[cat].astype('category')
  data_copy[cat] = dataset[cat].cat.codes

In [ ]:
train_dataset = data_copy.sample(frac=0.8, random_state=0)
test_dataset = data_copy.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('expenses')

test_labels = test_features.pop('expenses')

In [ ]:
normalizer = tf.keras.layers.Normalization() #axis=-1
normalizer.adapt(np.array(train_features))

model = tf.keras.Sequential([
    normalizer,
    layers.Dense(64, activation='tanh'),
    layers.Dense(64, activation='relu'),
    layers.Dense(units=1)
])

model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error',
    metrics=["mae",'mse'])

In [ ]:
%%time
history = model.fit(
    train_features,
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=True,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
print(hist.tail())

def plot_loss(history):
  plt.plot(history.history['mae'], label='mae')
  plt.plot(history.history['val_mae'], label='val_mae')
  #plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('mae')
  plt.legend()
  plt.grid(True)

plot_loss(history)

In [ ]:
# remove 'expenses' column from test_dataset
test_dataset = test_features

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
